In [1]:
import sys
sys.path.append('../')
import nbimporter
from basic.VaporPressure import rh_to_ah
from library.Temperature import Temperature

import matplotlib.pyplot as plt

Importing Jupyter notebook from ..\basic\VaporPressure.ipynb
Importing Jupyter notebook from ..\basic\SaturatedVaporPressure.ipynb


In [18]:
rh_to_ah(50.0, 31.0, 'SONNTAG','water')

OverflowError: math range error

In [3]:
class Air:
    def __init__(self,temp,hum,humtype,svpeq):
        self.svpeq = svpeq
        self.temp = temp
        self.ah   = {
            'relative' : rh_to_ah(hum,temp,svpeq,'water'),
            'absolute' : hum
        }[humtype]


In [4]:
Air(35.0,50,'relative','SONNTAG')

OverflowError: math range error